# Inspect best model

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *

from matplotlib.colors import LinearSegmentedColormap

from captum.attr import Occlusion
from captum.attr import IntegratedGradients
from captum.attr import GradientShap
from captum.attr import visualization as viz

In [3]:
CONFIG = {
    "dir_dataset": Path("./clean_dataset"),
    "dir_images": Path("./clean_dataset/images"),
    "trained_weights": "resnet34_unfreeze_img_size_252_DDD-36",
    "validation_set": "valid.txt",
    "model": models.resnet34,
    "metric": [error_rate, accuracy],
    "img_size": 252,
    "batch_size": 64,
}

In [4]:
# function for getting label information out of train file name
# Example: p002_c0_img_3370 -> c0
get_y_fn = lambda x: x.stem.split("_")[1]

In [5]:
# create Databunch
data = (ImageList.from_folder(CONFIG["dir_dataset"])
        .split_by_fname_file(CONFIG["validation_set"])
        .label_from_func(get_y_fn)
        .transform(get_transforms(do_flip=False), size=CONFIG["img_size"])
        .databunch(bs=CONFIG["batch_size"])
        .normalize(imagenet_stats)
    )

In [6]:
learn = cnn_learner(data, CONFIG["model"], metrics=CONFIG["metric"])

In [7]:
learn.load(CONFIG["trained_weights"]) ;

# Inspect examples

In [8]:
def integrated_gradients_plot(model, input_img, target, cmap, n_steps=50):
    integrated_gradients = IntegratedGradients(model)
    attributions_ig = integrated_gradients.attribute(input_img,
                                                     target=target, n_steps=n_steps)

    _ = viz.visualize_image_attr(np.transpose(attributions_ig.squeeze().cpu().detach().numpy(), (1,2,0)),
                                 np.transpose(input_img.squeeze().cpu().detach().numpy(), (1,2,0)),
                                 method='heat_map',
                                 cmap=cmap,
                                 show_colorbar=True,
                                 sign='positive',
                                 outlier_perc=1)

In [9]:
def gradient_shap_plot(model, input_img, target, cmap, n_samples=50):
    torch.manual_seed(0)
    np.random.seed(0)

    gradient_shap = GradientShap(model)

    # Defining baseline distribution of images
    rand_img_dist = torch.cat([input_img * 0, input_img * 1])

    attributions_gs = gradient_shap.attribute(input_img,
                                              n_samples=n_samples,
                                              stdevs=0.0001,
                                              baselines=rand_img_dist,
                                              target=target)
    
    _ = viz.visualize_image_attr_multiple(np.transpose(attributions_gs.squeeze().cpu().detach().numpy(), (1,2,0)),
                                          np.transpose(input_img.squeeze().cpu().detach().numpy(), (1,2,0)),
                                          ["original_image", "heat_map"],
                                          ["all", "absolute_value"],
                                          cmap=cmap,
                                          show_colorbar=True)

In [10]:
def occlusion_plot(model, input_img, target):
    occlusion = Occlusion(learn.model)

    rand_img_dist = torch.cat([input_img * 0, input_img * 1])
    attributions_occ = occlusion.attribute(input_img,
                                           strides = (3, 50, 50),
                                           target=target,
                                           sliding_window_shapes=(3,60, 60),
                                           baselines=0)

    _ = viz.visualize_image_attr_multiple(np.transpose(attributions_occ.squeeze().cpu().detach().numpy(), (1,2,0)),
                                          np.transpose(input_img.squeeze().cpu().detach().numpy(), (1,2,0)),
                                          ["original_image", "heat_map"],
                                          ["all", "positive"],
                                          show_colorbar=True,
                                          outlier_perc=2,)